# Database Read and Write Examples for R

Notes:
1. Be consistent with naming covention of your db tables. It is case sensitive.
2. RJDBC: https://cran.r-project.org/web/packages/RJDBC/RJDBC.pdf
3. Typo in [docs for RMySQL](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_latest/wsj/analyze-data/rstudio-rmysql-lib.html): which is a legacy package anyways and has been replaced by https://rmariadb.r-dbi.org/. Correction: libaray ('RMySQL') => library("RMySQL")
4. This notebook has been tested in R 3.6 Notebook in CPD 3.5.

<a class="anchor" id="toc"></a>
## TOC:
1. [Create db connection using RJDBC](#dbcon)  
* [Option 1: Use Insert to Code](#inser-to-code) 
2. [Common SQL Actions](#sql)
* [2.1 Query tables in three steps: write, send, fetch](#sql1)
* [2.2 Write tables](#sql2)
* [2.3 Update tables](#sql3)
* [2.4 Delete tables](#sql4)

3. [Close db connection](#close)   

4. [Appendix: Read and save data to local project](#projectlib)

<a class="anchor" id="dbcon"></a>
## 1. Create db connection using RJDBC
[Top](#toc)


<a class="anchor" id="inser-to-code"></a>
### Option 1: Use Insert to Code 

**The following cell was generated by Insert to Code(R DataFrame) menu. You can either regenerate it for Connected Data defined in your project or modify this code.**

**Important:**
1. Code generation does not always generate the first part of required code (creating a connection) if other generated code is already in the notebook. This is a bug. If you see this issue, generate the code in a new notebook and paste it into this one.* 
2. We had to add *identifier.quote = "\""* to the code that creates the driver (starts with *drv <-*). If you do not add this line, database write to DB2 throws an error. 

If you are not regenerating code, make sure to verify or change:

1. Connection name
2. Schema name
3. Table name

*Look for "To Do" tag in the code*

In [ ]:
# @hidden_cell

library(projectLib)
project <- projectLib::Project$new()

# To Do: change or verify connection name. It should match the connection name that you defined in your project
DB2_Cloud_metadata = project$get_connection(name="DB2 Cloud")

library(RJDBC)

drv <- JDBC(driverClass="com.ibm.db2.jcc.DB2Driver", classPath="/opt/jdbc/db2jcc4.jar",identifier.quote = "\"")

DB2_Cloud_url <- paste("jdbc:db2://",
    DB2_Cloud_metadata[][["host"]],
    ":", "50000",
    "/", DB2_Cloud_metadata[][["database"]],
    sep=""
)

DB2_Cloud_connection <- dbConnect(drv,
    DB2_Cloud_url,
    DB2_Cloud_metadata[][["username"]],
    DB2_Cloud_metadata[][["password"]]
)

# To Do: change to your own table
#query <- "SELECT * FROM \"TLR48624\".\"CUSTOMER_CHURN\""
query <- "SELECT * FROM TLR48624.CUSTOMER_CHURN"
data <- dbSendQuery(DB2_Cloud_connection, query)
# fetch first 5 rows
data_df_1 <- dbFetch(data, n = 5)
head(data_df_1)

# After use, close the database connection with the following code:
# dbDisconnect(ELDB2_connection)


 <a class="anchor" id="sql"></a>
## 2. Common SQL Actions
[Top](#toc)


 <a class="anchor" id="sql1"></a>
### 2.1 Query tables in three steps: write, send, fetch

In [ ]:
# To Do: change to your own table
# write SQL query
query <- "SELECT * FROM TLR48624.CUSTOMER_CHURN"
# send query
data <- dbSendQuery(DB2_Cloud_connection, query)

# fetch results
data_df_1 <- dbFetch(data)

head(data_df_1)

 <a class="anchor" id="sql2"></a>
### 2.2 Write tables
[Top](#toc)

Using iris as an example.

In [ ]:
data(iris)

In [ ]:
# To Do: change to your own table
# write tables
dbWriteTable(DB2_Cloud_connection, "TLR48624.RTABLE", iris)

In [ ]:
# To Do: change to your own table
# check if table has been created successfully
# write SQL query
query <- "SELECT * FROM TLR48624.RTABLE"

# send query
data <- dbSendQuery(DB2_Cloud_connection, query)

# fetch results
data_df_1 <- dbFetch(data)

head(data_df_1)

 <a class="anchor" id="sql3"></a>
## 2.3 Update tables
[Top](#toc)

### 1. using **dbSendUpdate**

In [ ]:
# To Do: change to your own table
# using dbSendUpdate
query <- "INSERT INTO TLR48624.RTABLE VALUES (5.1,3.5,1.4, 0.2, 'test')"
data <- dbSendUpdate(DB2_Cloud_connection, query)

In [ ]:
# To Do: change to your own table
# check if insert works
query <- "SELECT * FROM TLR48624.RTABLE WHERE SPECIES = 'test'"
data <- dbSendQuery(DB2_Cloud_connection, query)
data_df_1 <- dbFetch(data)
data_df_1

### 2. using **dbWriteTable** with append option

In [ ]:
dim(iris)

In [ ]:
# To Do: change to your own table
# Note: dont include schema for this line aka. use RTABLE instead of jpf66625.RTABLE
dbWriteTable(DB2_Cloud_connection, 'RTABLE', iris, append = TRUE, overwrite= F)

In [ ]:
# To Do: change to your own table
# check if append works: it should be 150 + 150 + 1 = 301
query <- "SELECT * FROM TLR48624.RTABLE"
data <- dbSendQuery(DB2_Cloud_connection, query)
data_df_1 <- dbFetch(data)
dim(data_df_1)

 <a class="anchor" id="sql4"></a>
## 2.4 Delete tables
[Top](#toc)


In [ ]:
# To Do: change to your own table
query <- "DROP TABLE TLR48624.RTABLE"
data <- dbSendUpdate(DB2_Cloud_connection, query)

In [ ]:
# To Do: change to your own table
# dont include schema for this line aka. use RTABLE instead of jpf66625.RTABLE
dbExistsTable(DB2_Cloud_connection, "RTABLE")

<a class="anchor" id="close"></a>
## 3. Close db connection
[Top](#toc)


In [ ]:
# After use, close the database connection with the following code:
dbDisconnect(DB2_Cloud_connection)

<a class="anchor" id="projectlib"></a>
## Appendix: saving to local project

[Top](#toc)

In [ ]:
library(projectLib)
#project <- projectLib::Project$new()
project <- access_project()

### Read data

In [ ]:
# To do - replace with the file name you want to test
my.file <- project$get_file("new_customers.csv")
df.data <-  read.csv(text = rawToChar(my.file))
head(df.data)

### Save data

In [ ]:
csv_lines <- capture.output(write.csv(df.data, row.names=FALSE), type="output")
csv_raw <- charToRaw(paste0(csv_lines, collapse='\n'))
project$save_data("file.csv", csv_raw)

**Written by Catherine Cao, April 2021** 